## 1. Introduction

This project implements a **GDPR-Compliant Hospital Management System** that upholds the CIA Triad:

- **Confidentiality**: Data encryption and role-based access control
- **Integrity**: Audit logging and data validation
- **Availability**: Stable system access and backup mechanisms

### Features Implemented:
- ✅ Role-Based Access Control (Admin, Doctor, Receptionist)
- ✅ Fernet Encryption for reversible data anonymization
- ✅ Data masking for sensitive identifiers
- ✅ Complete audit trail system
- ✅ GDPR consent management
- ✅ Data retention automation
- ✅ Real-time activity analytics
- ✅ CSV export for data portability

## 2. System Requirements

Install the required packages:

In [ ]:
# Install required packages
# Run this in terminal: pip install -r requirements.txt

# Required packages:
# - streamlit: Web application framework
# - pandas: Data manipulation
# - plotly: Interactive visualizations
# - cryptography: Fernet encryption for data security

!pip install streamlit pandas plotly cryptography

## 3. Database Setup

The system uses SQLite with the following schema:

In [ ]:
# Database Schema Overview

# Table 1: users
# - user_id (PRIMARY KEY)
# - username (UNIQUE)
# - password (SHA-256 hashed)
# - role (admin, doctor, receptionist)
# - created_at (TIMESTAMP)

# Table 2: patients
# - patient_id (PRIMARY KEY)
# - name, contact, diagnosis (Original data)
# - anonymized_name, anonymized_contact (Masked data)
# - encrypted_name, encrypted_contact, encrypted_diagnosis (Fernet encrypted)
# - date_added, is_anonymized, consent_given, data_retention_date

# Table 3: logs
# - log_id (PRIMARY KEY)
# - user_id, username, role
# - action, timestamp, details

# Table 4: consent_records
# - consent_id (PRIMARY KEY)
# - patient_id (FOREIGN KEY)
# - consent_type, consent_given, consent_date

print("Database schema defined in database.py")
print("All tables include proper constraints and foreign keys")

### Initialize Database

The database is automatically initialized when the application starts:

In [ ]:
# Import database manager
from database import DatabaseManager

# Initialize database
db = DatabaseManager('hospital_management.db')

print("✅ Database initialized successfully")
print("✅ Default users created:")
print("   - admin / admin123 (Admin)")
print("   - DrBob / doc123 (Doctor)")
print("   - Alice_recep / rec123 (Receptionist)")
print("✅ Sample patient records added")
print("✅ Encryption key generated")

## 4. Confidentiality Implementation

### 4.1 Password Hashing

In [ ]:
import hashlib

# Example: Password hashing for user authentication
def hash_password(password):
    """Hash password using SHA-256"""
    return hashlib.sha256(password.encode()).hexdigest()

# Demonstration
password = "admin123"
hashed = hash_password(password)

print(f"Original password: {password}")
print(f"Hashed password: {hashed}")
print(f"\n✅ Passwords are never stored in plain text")

### 4.2 Fernet Encryption (Reversible)

In [ ]:
from cryptography.fernet import Fernet

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)

# Example: Encrypting patient data
patient_name = "John Smith"
patient_diagnosis = "Hypertension"

# Encrypt
encrypted_name = cipher.encrypt(patient_name.encode())
encrypted_diagnosis = cipher.encrypt(patient_diagnosis.encode())

print("Original Data:")
print(f"  Name: {patient_name}")
print(f"  Diagnosis: {patient_diagnosis}")
print("\nEncrypted Data:")
print(f"  Name: {encrypted_name}")
print(f"  Diagnosis: {encrypted_diagnosis}")

# Decrypt
decrypted_name = cipher.decrypt(encrypted_name).decode()
decrypted_diagnosis = cipher.decrypt(encrypted_diagnosis).decode()

print("\nDecrypted Data:")
print(f"  Name: {decrypted_name}")
print(f"  Diagnosis: {decrypted_diagnosis}")
print("\n✅ Fernet encryption allows reversible anonymization")

### 4.3 Data Masking

In [ ]:
# Example: Data masking for display

def mask_name(name, patient_id):
    """Mask patient name with anonymous identifier"""
    return f"ANON_{patient_id:04d}"

def mask_contact(contact):
    """Mask contact number, showing only last 4 digits"""
    if len(contact) >= 4:
        return "XXX-XXX-" + contact[-4:]
    return "XXX-XXX-XXXX"

# Demonstration
original_name = "John Smith"
original_contact = "555-123-4567"
patient_id = 1021

masked_name = mask_name(original_name, patient_id)
masked_contact = mask_contact(original_contact)

print("Original Data:")
print(f"  Name: {original_name}")
print(f"  Contact: {original_contact}")
print("\nMasked Data:")
print(f"  Name: {masked_name}")
print(f"  Contact: {masked_contact}")
print("\n✅ Data masking protects patient identity")

### 4.4 Role-Based Access Control (RBAC)

In [ ]:
# RBAC Implementation

roles_permissions = {
    'admin': {
        'can_view_raw_data': True,
        'can_view_anonymized': True,
        'can_add_patient': True,
        'can_edit_patient': True,
        'can_delete_patient': True,
        'can_anonymize': True,
        'can_view_logs': True,
        'can_export_data': True
    },
    'doctor': {
        'can_view_raw_data': False,
        'can_view_anonymized': True,
        'can_add_patient': False,
        'can_edit_patient': False,
        'can_delete_patient': False,
        'can_anonymize': False,
        'can_view_logs': False,
        'can_export_data': False
    },
    'receptionist': {
        'can_view_raw_data': False,
        'can_view_anonymized': True,
        'can_add_patient': True,
        'can_edit_patient': True,
        'can_delete_patient': False,
        'can_anonymize': False,
        'can_view_logs': False,
        'can_export_data': False
    }
}

# Display permissions
import pandas as pd

df_permissions = pd.DataFrame(roles_permissions).T
print("Role-Based Access Control Matrix:")
print(df_permissions)
print("\n✅ Each role has specific permissions")

## 5. Integrity Implementation

### 5.1 Audit Logging System

In [ ]:
# Example: Logging user actions

from datetime import datetime

def log_action_example(user_id, username, role, action, details):
    """
    Log user action for audit trail
    All actions are recorded with timestamp
    """
    log_entry = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'user_id': user_id,
        'username': username,
        'role': role,
        'action': action,
        'details': details
    }
    return log_entry

# Example logs
log1 = log_action_example(1, 'admin', 'admin', 'login', 'User admin logged in')
log2 = log_action_example(1, 'admin', 'admin', 'anonymize_data', 'Anonymized 5 patient records')
log3 = log_action_example(2, 'DrBob', 'doctor', 'view_patients', 'Viewed 5 patient records')

print("Sample Audit Logs:")
for i, log in enumerate([log1, log2, log3], 1):
    print(f"\nLog {i}:")
    for key, value in log.items():
        print(f"  {key}: {value}")

print("\n✅ All actions are logged for accountability")

### 5.2 Data Validation

In [ ]:
# Example: Input validation for patient data

import re

def validate_patient_data(name, contact, diagnosis):
    """
    Validate patient data before insertion
    Ensures data integrity
    """
    errors = []
    
    # Validate name
    if not name or len(name.strip()) < 2:
        errors.append("Name must be at least 2 characters")
    
    # Validate contact (simple pattern)
    if not contact or not re.match(r'^[0-9\-\s\+\(\)]+$', contact):
        errors.append("Invalid contact number format")
    
    # Validate diagnosis
    if not diagnosis or len(diagnosis.strip()) < 3:
        errors.append("Diagnosis must be at least 3 characters")
    
    return len(errors) == 0, errors

# Test validation
test_cases = [
    ("John Smith", "555-123-4567", "Hypertension"),
    ("", "555-123-4567", "Hypertension"),
    ("John", "invalid", "H")
]

for name, contact, diagnosis in test_cases:
    valid, errors = validate_patient_data(name, contact, diagnosis)
    print(f"\nData: {name}, {contact}, {diagnosis}")
    print(f"Valid: {valid}")
    if errors:
        print(f"Errors: {errors}")

print("\n✅ Data validation prevents invalid entries")

## 6. Availability Implementation

### 6.1 Error Handling

In [ ]:
# Example: Robust error handling

def safe_database_operation():
    """
    Example of error handling for database operations
    Ensures system remains available even on errors
    """
    try:
        # Simulated database operation
        result = "Operation successful"
        return True, result
    
    except Exception as e:
        # Log error and return graceful failure
        error_msg = f"Database error: {str(e)}"
        print(f"ERROR: {error_msg}")
        return False, error_msg

# Test error handling
success, message = safe_database_operation()
print(f"Success: {success}")
print(f"Message: {message}")
print("\n✅ Error handling ensures system stability")

### 6.2 Data Backup & Export

In [ ]:
# Example: CSV export for data backup

import csv
import io

def export_to_csv(data, headers):
    """
    Export data to CSV format for backup
    Implements GDPR right to data portability
    """
    output = io.StringIO()
    writer = csv.writer(output)
    
    # Write headers
    writer.writerow(headers)
    
    # Write data
    for row in data:
        writer.writerow(row)
    
    return output.getvalue()

# Sample data export
sample_data = [
    [1, 'ANON_0001', 'XXX-XXX-4567', '[ENCRYPTED]', '2025-11-17', True],
    [2, 'ANON_0002', 'XXX-XXX-6543', '[ENCRYPTED]', '2025-11-17', True]
]
headers = ['ID', 'Name', 'Contact', 'Diagnosis', 'Date', 'Anonymized']

csv_output = export_to_csv(sample_data, headers)
print("CSV Export:")
print(csv_output)
print("✅ Data can be exported for backup and recovery")

## 7. Bonus Features

### 7.1 Real-Time Activity Graphs

In [ ]:
# Example: Activity analytics data preparation

import pandas as pd
from datetime import datetime, timedelta

# Sample activity data (last 7 days)
dates = [(datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(6, -1, -1)]
activities = [12, 15, 10, 18, 22, 20, 25]

df_activity = pd.DataFrame({
    'Date': dates,
    'Actions': activities
})

print("Daily Activity Summary:")
print(df_activity)
print("\n✅ Activity data visualized in real-time graphs")
print("   (Charts rendered in Streamlit dashboard using Plotly)")

### 7.2 Data Retention Timer

In [ ]:
# Example: Data retention calculation

from datetime import datetime, timedelta

def calculate_retention_date(creation_date, retention_days=30):
    """
    Calculate when data should be deleted (GDPR compliance)
    Default retention: 30 days
    """
    return creation_date + timedelta(days=retention_days)

def check_expiry(retention_date):
    """
    Check if data has expired
    """
    return datetime.now() > retention_date

# Example
created = datetime(2025, 10, 15)
retention_date = calculate_retention_date(created, 30)
is_expired = check_expiry(retention_date)

print(f"Record created: {created.strftime('%Y-%m-%d')}")
print(f"Retention date: {retention_date.strftime('%Y-%m-%d')}")
print(f"Is expired: {is_expired}")
print("\n✅ Automated data retention ensures GDPR compliance")

### 7.3 GDPR Consent Management

In [ ]:
# Example: Consent tracking

def record_consent(patient_id, consent_type, consent_given):
    """
    Record patient consent for data processing
    Implements GDPR right to be informed
    """
    consent_record = {
        'patient_id': patient_id,
        'consent_type': consent_type,
        'consent_given': consent_given,
        'consent_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    return consent_record

# Example consent records
consent1 = record_consent(1, 'data_processing', True)
consent2 = record_consent(2, 'data_processing', True)

print("Consent Records:")
for consent in [consent1, consent2]:
    print(f"\nPatient {consent['patient_id']}:")
    print(f"  Type: {consent['consent_type']}")
    print(f"  Given: {consent['consent_given']}")
    print(f"  Date: {consent['consent_date']}")

print("\n✅ Consent tracking ensures GDPR compliance")

## 8. Running the Application

### Step-by-Step Guide

In [ ]:
# Step 1: Install dependencies
# Run in terminal: pip install -r requirements.txt

# Step 2: Initialize database
# The database is automatically created when you run the app

# Step 3: Run the Streamlit application
# Run in terminal: streamlit run app.py

# Step 4: Access the application
# Open browser at: http://localhost:8501

print("\n" + "="*60)
print("RUNNING THE APPLICATION")
print("="*60)
print("\n1. Install dependencies:")
print("   pip install -r requirements.txt")
print("\n2. Run the application:")
print("   streamlit run app.py")
print("\n3. Login credentials:")
print("   Admin: admin / admin123")
print("   Doctor: DrBob / doc123")
print("   Receptionist: Alice_recep / rec123")
print("\n4. Access at: http://localhost:8501")
print("="*60)

## System Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    PRESENTATION LAYER                       │
│              Streamlit Web Application (app.py)             │
│  ┌──────────┐  ┌──────────┐  ┌──────────────────────────┐  │
│  │  Login   │  │   Admin  │  │  Doctor/Receptionist     │  │
│  │  Page    │  │Dashboard │  │     Dashboards          │  │
│  └──────────┘  └──────────┘  └──────────────────────────┘  │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│                    BUSINESS LOGIC LAYER                     │
│               Database Manager (database.py)                │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────────┐  │
│  │     RBAC     │  │  Encryption  │  │  Audit Logging   │  │
│  │              │  │   (Fernet)   │  │                  │  │
│  └──────────────┘  └──────────────┘  └──────────────────┘  │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│                      DATA LAYER                             │
│              SQLite Database (hospital_management.db)       │
│  ┌──────────┐  ┌──────────┐  ┌──────┐  ┌──────────────┐   │
│  │  users   │  │ patients │  │ logs │  │   consent    │   │
│  │  table   │  │  table   │  │table │  │   records    │   │
│  └──────────┘  └──────────┘  └──────┘  └──────────────┘   │
└─────────────────────────────────────────────────────────────┘
```

## CIA Triad Implementation Summary

### 🔒 Confidentiality
1. **Password Hashing**: SHA-256 hashing for user passwords
2. **Fernet Encryption**: Reversible encryption for patient data
3. **Data Masking**: Anonymized identifiers (ANON_XXXX, XXX-XXX-XXXX)
4. **Role-Based Access Control**: Admin, Doctor, Receptionist roles
5. **Secure Authentication**: Login system with session management

### ✅ Integrity
1. **Audit Logging**: Complete trail of all user actions
2. **Data Validation**: Input validation prevents invalid entries
3. **Database Constraints**: Foreign keys and check constraints
4. **Timestamps**: All records include creation/modification times
5. **Admin-Only Modifications**: Critical actions restricted to admin

### 🚀 Availability
1. **Error Handling**: Try-except blocks for graceful failures
2. **Data Backup**: CSV export functionality
3. **Stable Architecture**: SQLite with connection pooling
4. **System Monitoring**: Activity logs and metrics
5. **Recovery Options**: Data export and restore capabilities

### 🎁 Bonus Features
1. **Fernet Encryption**: Reversible anonymization
2. **Real-Time Analytics**: Interactive Plotly graphs
3. **GDPR Features**:
   - Data retention timer (30 days)
   - Consent banner and tracking
   - Right to erasure (delete records)
   - Right to data portability (CSV export)
   - Right to be informed (privacy notices)

## Testing the System

### Test Scenarios

In [ ]:
# Test Scenario 1: Admin Login and Data Anonymization
print("TEST SCENARIO 1: Admin Workflow")
print("=" * 50)
print("1. Login as admin (admin/admin123)")
print("2. View raw patient data")
print("3. Click 'Anonymize All Patient Data'")
print("4. Verify data is encrypted")
print("5. Check audit logs for anonymization action")
print("6. Export audit logs as CSV")
print("\nExpected Result: All patient data encrypted with Fernet")
print()

# Test Scenario 2: Doctor Access
print("TEST SCENARIO 2: Doctor Workflow")
print("=" * 50)
print("1. Login as doctor (DrBob/doc123)")
print("2. View patient list (anonymized only)")
print("3. Verify cannot edit or delete records")
print("4. Check that raw data is not visible")
print("\nExpected Result: Only anonymized data visible, no edit rights")
print()

# Test Scenario 3: Receptionist Operations
print("TEST SCENARIO 3: Receptionist Workflow")
print("=" * 50)
print("1. Login as receptionist (Alice_recep/rec123)")
print("2. Add new patient with consent")
print("3. Edit existing patient record")
print("4. Verify cannot delete records")
print("5. Check that action is logged")
print("\nExpected Result: Can add/edit but not delete, all logged")
print()

# Test Scenario 4: GDPR Compliance
print("TEST SCENARIO 4: GDPR Compliance")
print("=" * 50)
print("1. Check consent banner appears on first visit")
print("2. View consent statistics in GDPR tab")
print("3. Run data retention cleanup")
print("4. Export patient data (data portability)")
print("5. Delete patient record (right to erasure)")
print("\nExpected Result: All GDPR rights implemented")
print()

## Conclusion

This Hospital Management System successfully implements:

✅ **All Core Requirements**
- SQLite database with proper schema
- Role-Based Access Control (RBAC)
- Data anonymization and masking
- Complete audit logging system
- Error handling and data backup

✅ **All Bonus Features**
- Fernet encryption (reversible)
- Real-time activity graphs
- Data retention timer
- User consent banner
- GDPR compliance tools

✅ **CIA Triad Compliance**
- **Confidentiality**: Encryption, RBAC, masking
- **Integrity**: Audit logs, validation, constraints
- **Availability**: Error handling, backup, stable access

### Project Files:
1. `database.py` - Database management and encryption
2. `app.py` - Streamlit web application
3. `requirements.txt` - Python dependencies
4. `Assignment4.ipynb` - This documentation notebook
5. `hospital_management.db` - SQLite database (auto-generated)
6. `encryption.key` - Fernet encryption key (auto-generated)

### Quick Start:
```bash
pip install -r requirements.txt
streamlit run app.py
```

**System is ready for demonstration and submission!** 🎉